<a href="https://colab.research.google.com/github/mars241/Machine-Learning/blob/main/Live_codingML_V4_de_W9_LC_ML_NearestNeighbor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

# Objectif : trouver qui sont les 10 clients qui ressemblent le plus au client 73

# 1. Import données

In [ ]:
from google.colab import  drive
drive.mount('/drive')

Mounted at /drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/livecoding/ML_df_customers.csv"
df = pd.read_csv(path)
df

,Education,Income,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Complain,age,nb_enf,situation_couple
0,Graduation,58138.0,635,88,546,172,88,88,1.0,2.0,2.0,1.0,1.0,0,65,0,celib
1,Graduation,46344.0,11,1,6,2,1,6,0.0,0.0,0.0,0.0,1.0,0,68,2,celib
2,Graduation,71613.0,426,49,127,111,21,42,0.0,2.0,0.0,2.0,1.0,0,57,0,couple
3,Graduation,26646.0,11,4,20,10,3,5,0.0,0.0,0.0,1.0,1.0,0,38,1,couple
4,PhD,58293.0,173,43,118,46,27,15,1.0,1.0,1.0,1.0,1.0,0,41,1,couple
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2003,Graduation,51412.0,9,14,18,8,1,12,1.0,1.0,0.0,1.0,1.0,0,45,1,couple
2004,Graduation,61223.0,709,43,182,42,118,247,0.0,2.0,1.0,1.0,1.0,0,55,1,couple
2005,Graduation,56981.0,908,48,217,32,12,24,0.0,0.0,1.0,3.0,1.0,0,41,0,celib
2006,Master,69245.0,428,30,214,80,30,61,0.0,1.0,1.0,2.0,1.0,0,66,1,couple


In [ ]:
df.index = ["client"+str(i) for i in df.index]
df.head()

,Education,Income,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Complain,age,nb_enf,situation_couple
client0,Graduation,58138.0,635,88,546,172,88,88,1.0,2.0,2.0,1.0,1.0,0,65,0,celib
client1,Graduation,46344.0,11,1,6,2,1,6,0.0,0.0,0.0,0.0,1.0,0,68,2,celib
client2,Graduation,71613.0,426,49,127,111,21,42,0.0,2.0,0.0,2.0,1.0,0,57,0,couple
client3,Graduation,26646.0,11,4,20,10,3,5,0.0,0.0,0.0,1.0,1.0,0,38,1,couple
client4,PhD,58293.0,173,43,118,46,27,15,1.0,1.0,1.0,1.0,1.0,0,41,1,couple


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2008 entries, client0 to client2007
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Education            2008 non-null   object 
 1   Income               2008 non-null   float64
 2   MntWines             2008 non-null   int64  
 3   MntFruits            2008 non-null   int64  
 4   MntMeatProducts      2008 non-null   int64  
 5   MntFishProducts      2008 non-null   int64  
 6   MntSweetProducts     2008 non-null   int64  
 7   MntGoldProds         2008 non-null   int64  
 8   NumDealsPurchases    2008 non-null   float64
 9   NumWebPurchases      2008 non-null   float64
 10  NumCatalogPurchases  2008 non-null   float64
 11  NumStorePurchases    2008 non-null   float64
 12  NumWebVisitsMonth    2008 non-null   float64
 13  Complain             2008 non-null   int64  
 14  age                  2008 non-null   int64  
 15  nb_enf               2008 non-n

# 2. Préparation des données

## 2.1 Encoder

In [ ]:
# FACTORIZE PAS ADAPTÉ
df["situation_couple"] = df["situation_couple"].factorize()[0]
df["niv_etudes"] = df["Education"].factorize()[0]

In [ ]:
# sparse = False permet de mieux utiliser des matrices pleines de 0 (mathématiquement + optimisé)
OHencod = OneHotEncoder(sparse=False, drop="if_binary")
df["en_couple"] = OHencod.fit_transform(df[["situation_couple"]])


In [ ]:
# catégories progressives, niveaux de diplômes sont pris en compte, PhD > Master > Basic
Ordencod = OrdinalEncoder(categories=[["Basic", "2n Cycle", "Graduation", "Master", "PhD"]])
df["niv_etudes"] = Ordencod.fit_transform(df[["Education"]])

In [ ]:
df.head()

,Education,Income,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Complain,age,nb_enf,situation_couple,en_couple
client0,Graduation,58138.0,635,88,546,172,88,88,1.0,2.0,2.0,1.0,1.0,0,65,0,celib,0.0
client1,Graduation,46344.0,11,1,6,2,1,6,0.0,0.0,0.0,0.0,1.0,0,68,2,celib,0.0
client2,Graduation,71613.0,426,49,127,111,21,42,0.0,2.0,0.0,2.0,1.0,0,57,0,couple,1.0
client3,Graduation,26646.0,11,4,20,10,3,5,0.0,0.0,0.0,1.0,1.0,0,38,1,couple,1.0
client4,PhD,58293.0,173,43,118,46,27,15,1.0,1.0,1.0,1.0,1.0,0,41,1,couple,1.0


In [ ]:
df["niv_etudes"].unique()

array([2., 4., 3., 1., 0.])

## 2.2 Données quanti

In [ ]:
X = df.select_dtypes("number")
X.head()

,Income,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Complain,age,nb_enf,en_couple
client0,58138.0,635,88,546,172,88,88,1.0,2.0,2.0,1.0,1.0,0,65,0,0.0
client1,46344.0,11,1,6,2,1,6,0.0,0.0,0.0,0.0,1.0,0,68,2,0.0
client2,71613.0,426,49,127,111,21,42,0.0,2.0,0.0,2.0,1.0,0,57,0,1.0
client3,26646.0,11,4,20,10,3,5,0.0,0.0,0.0,1.0,1.0,0,38,1,1.0
client4,58293.0,173,43,118,46,27,15,1.0,1.0,1.0,1.0,1.0,0,41,1,1.0


# 3. Modèle

In [ ]:
modelNN = NearestNeighbors()

In [ ]:
modelNN.fit(X)

NearestNeighbors()

In [ ]:
# Localiser le client73
indiv_concerne = X.loc["client73"].to_frame().T
indiv_concerne

,Income,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Complain,age,nb_enf,en_couple
client73,26326.0,1.0,0.0,5.0,0.0,3.0,9.0,0.0,0.0,0.0,1.0,1.0,1.0,46.0,0.0,1.0


In [ ]:
# On regarde les proches voisins
neigh_dist, neigh_cli = modelNN.kneighbors(indiv_concerne, n_neighbors=11)

In [ ]:
A = modelNN.kneighbors(indiv_concerne, n_neighbors=11)

In [ ]:
A

(array([[  0.        ,  24.18677324,  44.64302857,  98.32598843,
         103.97114984, 162.72676485, 177.61193654, 185.90857968,
         196.34153916, 231.82536531, 237.62365202]]),
 array([[  73,   76, 1352, 1149,  577, 1963,  585,  244,  666,  497,  162]]))

In [ ]:
# On regarde la distance des proches voisins
neigh_dist

array([[  0.        ,  24.18677324,  44.64302857,  98.32598843,
        103.97114984, 162.72676485, 177.61193654, 185.90857968,
        196.34153916, 231.82536531, 237.62365202]])

In [ ]:
# On regarde l'index
neigh_cli

array([[  73,   76, 1352, 1149,  577, 1963,  585,  244,  666,  497,  162]])

In [ ]:
cli_ressem = neigh_cli[0][1:]

In [ ]:
X.iloc[cli_ressem]

,Income,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Complain,age,nb_enf,en_couple
client76,26304.0,4,1,5,2,0,4,0.0,0.0,0.0,0.0,2.0,0,53,1,1.0
client1352,26290.0,15,8,16,11,5,22,1.0,0.0,0.0,1.0,1.0,0,50,2,1.0
client1149,26228.0,2,6,4,3,1,9,0.0,0.0,0.0,0.0,2.0,0,49,1,1.0
client577,26224.0,4,7,15,13,9,15,1.0,0.0,0.0,1.0,1.0,0,44,1,1.0
client1963,26487.0,2,8,10,12,14,23,1.0,0.0,0.0,1.0,1.0,0,44,1,0.0
client585,26150.0,5,1,13,3,5,1,0.0,0.0,0.0,1.0,2.0,0,66,3,0.0
client244,26490.0,45,47,52,21,20,25,0.0,1.0,0.0,1.0,1.0,0,64,0,1.0
client666,26518.0,20,1,28,3,3,2,1.0,0.0,0.0,1.0,2.0,0,73,2,0.0
client497,26095.0,11,7,9,3,1,11,0.0,0.0,0.0,1.0,2.0,0,32,1,0.0
client162,26091.0,15,10,19,8,17,20,1.0,0.0,0.0,1.0,1.0,0,65,2,1.0


In [ ]:
indiv_concerne

,Income,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Complain,age,nb_enf,situation_couple,niv_etudes,en_couple
clientclientclient73,26326.0,1.0,0.0,5.0,0.0,3.0,9.0,0.0,0.0,0.0,1.0,1.0,1.0,46.0,0.0,1.0,1.0,1.0
